In [10]:
import pyupbit
import time
import datetime
from pyupbit import get_current_price, get_tickers
# , get_ohlcv, get_daily_ohlcv_from_base
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def get_k_list(start=0.05, diff=0.05):
    out = []
    while(start < 1):
        out.append(round(start, 2))
        start += diff

    return out

In [3]:
def get_ohlcv(ticker, days):
    return pyupbit.get_ohlcv(ticker, count=days)

In [4]:
def get_ohlcv_base(ticker, days, base):
    df = pyupbit.get_ohlcv(ticker, interval="minute60", count = days*24)
    return df.resample('24H', base=base).agg({
        'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum', 'value': 'sum'
    })

In [5]:
# 단기 변동성 돌파 전략 v1
# 변동성 = 전일 고가 - 전일 저가
# 매수 목표 가격 = 변동성 * K
# 당일 시세가 목표 가격에 도달하면 매수
# 당일 종가에 매도

def vbvds_expect_rate_v1(df ,k):
    df['diff'] = df['high'] - df['low']
    df['target'] = df['diff'].shift(1) * k + df['open'] 
    df['bought'] = df['high'] > df['target']

    cond = df['high'] > df['target']
    df['rate'] = df.loc[cond, 'close'] / df.loc[cond, 'target']

    df['rate'].fillna(1, inplace=True)

    result = 1 
    for i in range(len(df)):
        result = result * df['rate'][i]

    return result

In [60]:
def expect_rate_v1_each_k(df):
    k_list = get_k_list(start=0.05, diff=0.05)
    rates = []
    for k in k_list:
        rate = vb_expect_rate_v1(df=df, k=k)
        rates.append(rate)
    
    out = pd.DataFrame({'rate': rates})
    out.index = k_list
    
    return out

KRW-BCH     -0.823713
KRW-WAVES   -0.755797
KRW-DOGE    -0.577054
KRW-ADA     -0.395888
KRW-THETA   -0.386611
               ...   
KRW-SOL      0.657595
KRW-XTZ      0.711160
KRW-KNC      0.729037
KRW-MANA     0.745020
target       1.000000
Name: target, Length: 111, dtype: float64